In [ ]:
"""给定一个字符串，请你找出其中不含有重复字符的 最长子串 的长度。"""

'给定一个字符串，请你找出其中不含有重复字符的 最长子串 的长度。'

In [1]:
"""
【思路】
substring：必须连续！
暴力法，遍历所有substring（N^2），对每个sbustring，遍历一遍来判断重复性，在所有没有重复的substring找最大的一个，时间复杂度O(N^3)，空间复杂度O(1)

假如一个substring已经判断没有重复，
那么由这个substring组成的新的更长的substring，还有必要再遍历一遍来判断重复性？
没必要，只要看新加入的元素是否在之前的substring中，通过这样的方法来对新的substring判断重复性（显然这样方法没有遍历整个新的substring，复杂度也降低了）。
那么，可以看到，我们要维护一个没有重复元素的substring，这样才能降低新的更长的substring的判断重复性的复杂度。
那么维护这么一个substring，其实就是一个“滑动窗口”。

首先我用的是这么一个方法：
1. 用列表ret维护一个不重复的substring：s[...i]， i初值为-1，ret的length就是滑动窗口的大小
2. 查看下一个元素i+1
3. 当s[i+1]存在于ret中，那么s[...i] + s[i+1]构成的substring肯定有重复元素（具体就是s[i+1]和前面某个值重复了）
此时，我们只要将 s[...i]从左边往右缩，不断的扔掉s[...i]最左边的元素，直到s[i+1] not in s[...i]， 此时s[...i] + s[i+1]构成的substring就肯定不会有重复元素了
4. 当s[i+1]不存在于ret中，s[...i] + s[i+1]构成的新的substring，无重复元素且更长。
5. 每次i的遍历都能由原来的substring和s[i+1]构成新的substring，记录其长度。当i遍历完了，最长的那个子串长度也就找到了。
"""


class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        # ret is a list contains non-repeated substring
        # [...i] is non-repeated
        i = -1
        max_len = 0
        ret = []
        while i < len(s):
            i += 1
            if i < len(s):
                if s[i] in ret:
                    while s[i] in ret:
                        ret.pop(0)
                ret.append(s[i])
            max_len = max(max_len, len(ret))
        return max_len


"""
【复杂度】
时间复杂度：i遍历n次，每次遍历中，判断s[i+1]是否在ret中，最坏需要n次，所以最坏复杂度是O(N^2)（扔掉元素不会增加复杂度的数量级）
空间复杂度：O(k), k就是ret的大小，k=min(n,m), n是string的长度，m是字符串的字符集的大小。
【复盘】
1. 滑动窗口写的太丑，明明可以用两个指针维护一个窗口，却用一个指针加列表来维护。增加空间复杂度
2. 时间复杂度还是太高，遍历窗口这个无法优化，但是判断元素在substring可否优化时间复杂度？
3. 体会下边界条件，维护的滑动窗口[...i]不重复, 不断查看i+1，生成新的以i+1结尾的substring [...i+1];回顾283 move zeros的边界辨析。
或者说维护的滑动窗口[...i+1)不重复    【循环变量先递增，再操作】
* [...i)不重复，那么直接查看i，再生成[...i]即[...i+1)   【循环变量，先操作，再递增】
"""

'\n【复杂度】\n时间复杂度：i遍历n次，每次遍历中，判断s[i]是否在ret中，最坏需要n次，所以最坏复杂度是O(N^2)（扔掉元素不会增加复杂度的数量级）\n空间复杂度：O(k), k就是ret的大小，k=min(n,m), n是string的长度，m是字符串的字符集的大小。\n【复盘】\n1. 滑动窗口写的太丑，明明可以用两个指针维护一个窗口，却用一个指针加列表来维护。增加空间复杂度\n2. 时间复杂度还是太高，遍历窗口这个无法优化，但是判断元素在substring可否优化时间复杂度？\n3. 体会下边界条件，维护的滑动窗口[...i-1]不重复, 先更新i++，，再生成新的substring [...i];回顾283 move zeros的边界辨析。\n或者说维护的滑动窗口[...i)不重复\n'

In [5]:
"""
【思路】
1. 更加规范的滑动窗口的写法
2. 时间复杂度仍然是O(N^2)， 但是空间复杂度是O(1)
3. 对比上例，体会边界条件
"""


class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        # [i, j) is non-repeated
        i, j = 0, 0
        max_len = 0
        while j < len(s):
            if s[j] not in s[i:j]:
                j += 1
            else:
                i += 1
            max_len = max(max_len, j-i)
        return max_len


In [5]:
"""
【思路】
通过hashset降低查找s[i]是否和滑动窗口中的元素重复的复杂度，只用O(1)时间
但是需要额外的空间O（k），其实是一个hashset的滑动窗口
1. 如果s[j]不在set中，加进去
2. 如果s[j]在set中，不断移除set中的s[i],使得s[j]和set没有相同元素

最终使得包含s[i..j]都在set中（维护了一个没有重复的hashset滑动窗口）

"""
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        # [i, j) is non-repeated
        i, j = 0, 0
        ss = set()
        max_len = 0
        while j < len(s):
            if s[j] not in ss:
                ss.add(s[j])
                j += 1
            else: # 一次判断
                ss.remove(s[i]) # 一次删除
                i += 1
            max_len = max(max_len, j-i)
        return max_len

In [20]:
"""
【优化】 使用hashmap的滑动数组
1. 如果s中重复数组较多，基本走下面的分支，有两个步骤，先用j判断O(1),再用i删除O(1)，
可能每个字符被访问两次
2. 用j遍历这个无法省略，i更新是通过不断判断s[j]是否存在set。可不可以一步得到i的更新？
使用hashmap，记录字符到索引的map，那么遇到重复的s[j]，直接更新i=map[s[j]] + 1
找到重复元素的索引加1即可

【错误】
1. 我之前写i更新公式： i = ss.pop(s[j]) + 1
2. 遇到这个"abba"测试用例，出错了！
3. 当j=3,i=2; j=4, i=1; 可以发现i竟然变小了，这种情况不允许发生，i只可以不变或增加
所以使用max来避免这种情况。上面hashset的滑动数组能轻易避免这种情况是因为，i不会变小

【可能优化思路】
1. 空间复杂度？如果字符集不大，可否用数组代替hashtable，比如字符集就是26个英文字母，
就用一个int[26]来代替hashtable


"""


class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        # [i, j) is non-repeated
        i, j = 0, 0
        ss = dict()
        max_len = 0
        while j < len(s):
            if s[j] not in ss:
                ss[s[j]] = j
                j += 1
            else:  # s[j] in ss
                i = max(ss.pop(s[j]) + 1, i)
            max_len = max(max_len, j - i)
        return max_len


In [21]:
ss = Solution()
ss.lengthOfLongestSubstring("abba")

2